In [1]:
import numpy as np
import pandas as pd 
from keras.utils import np_utils
import os
import keras
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.pyplot as plt
from numpy import array
from keras.utils import to_categorical
from numpy import argmax
from keras import Sequential
from keras.layers import LSTM,Input,Lambda,concatenate, Dense,Dropout,GRU,BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text as keras_text, sequence as keras_seq
import re
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adadelta
from time import time
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import tensorflow as tf
from keras import optimizers

Using TensorFlow backend.


In [6]:
print(os.listdir("../input/testcs/train.csv"))

['train.csv']


In [ ]:
def ReadData():
    train1 = pd.read_csv("../input/testcs/train.csv",encoding ='latin1')
    test1 = pd.read_csv("../input/trainc/tratestin.csv",encoding ='latin1')
    desc1 = pd.read_csv("../input/product_descriptions.csv",encoding ='latin1')
    sample1 = pd.read_csv("../input/sample_submission.csv",encoding ='latin1')
    attributes1 = pd.read_csv("../input/attributes.csv",encoding ='latin1')
    return train1,test1,desc1,sample1,attributes1
def ConcatDescAndTitle(df):
    train1 = df.merge(desc,on='product_uid',how='left')
    #train1['TitleAndDesc'] = train1[['product_title', 'product_description']].apply(lambda x: ''.join(x), axis=1)
    train1 = train1.drop(columns=['product_description','id'])
    #train1['product_title'] = train1['product_title'].str.replace('[^a-zA-Z0-9\s]','$')
    #train1['search_term'] = train1['search_term'].str.replace('[^a-zA-Z0-9\s]','$')
    train1['product_title'] = train1['product_title'].str.replace('\\xa0', '')
    train1['search_term'] = train1['search_term'].str.replace('\\xa0', '')
    return train1
def LeftRightAndY(df):
    left = df["product_title"]
    right = df["search_term"]
    y = df["relevance"]
    return left,right,y
def getBrandAndMatirial():
    for i in range(train.shape[0]):
        if (i%1000==0):
            print(i)
        row = train.loc[i]
        #foo = df.ix[(df['column1']==value) | (df['columns2'] == 'b') | (df['column3'] == 'c')]
        row2 = attributes[attributes["product_uid"]==row["product_uid"]]
        brand = row2[row2["name"]==str("MFG Brand Name")]["value"]
        Material = row2[row2["name"]==str("Material")]["value"]
        if ((brand.values.any())):
            if ((Material.values.any())):
                train.loc[i,"product_title"] += " " + str(brand.values[0]) + " " + str(Material.values[0])
    return train

In [ ]:
train,test,desc,sample,attributes = ReadData()
train = ConcatDescAndTitle(train)
test = ConcatDescAndTitle(test)
train = getBrandAndMatirial()
#input_left,input_right,y = LeftRightAndY(train)

In [ ]:
train.to_csv('train.csv',index=False)
test.to_csv('tratestin.csv',index=False)

In [ ]:
def CreateDic():
    CharToIndex2 = {}
    counter=0
    for key in model.wv.vocab.keys():
        CharToIndex2[key] = counter
        counter += 1
    return CharToIndex2
def CreateOneHot(index):
    arr = np.zeros(lenthofchar)
    arr[index] = 1
    return arr

In [ ]:
y_train=train["relevance"]
until = train.shape[0]*0.85
X_train = train.loc[:until,:]
X_test = train.loc[until:,:]
Y_train = y_train.loc[:until]
Y_test = y_train.loc[until:]
avg = 0
chars = "abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
for i in range(X_train.shape[0]):
    lis =(list(X_train.loc[i]["product_title"]))
    avg += len(lis)
avg_len = avg/X_train.shape[0]
avg_len =int(math.floor(avg_len))
avg_len=60
lenthofchar = len(chars)
x_train_left = np.empty([X_train.shape[0],avg_len,lenthofchar])
for i in range(X_train.shape[0]):
    lis =(list(X_train.loc[i]["product_title"]))
    for j in range(avg_len):
        if (j<len(lis)):
            x_train_left[i][j] = CreateOneHot(chars.find(lis[j]))
        else:
            x_train_left[i][j] = CreateOneHot(chars.find("$"))
x_train_right = np.empty([X_train.shape[0],avg_len,lenthofchar])
for i in range(X_train.shape[0]):
    lis2 =(list(X_train.loc[i]["search_term"]))
    for j in range(avg_len):
        if (j<len(lis2)):
            x_train_right[i][j] = CreateOneHot(chars.find(lis2[j]))
        else:
            x_train_right[i][j] = CreateOneHot(chars.find("$"))

In [ ]:
def cossim(left, right):
    ans = (K.sum(left*right))/(K.sqrt(K.sum(K.pow(left,2)))*K.sqrt(K.sum(K.pow(right,2))))
    ans = tf.nn.relu(ans)
    return ans*2 +1

In [ ]:
y_train=train["relevance"]
Y_train = y_train.loc[:until]
Y_test = y_train.loc[until:]
Y_train = np_utils.to_categorical(Y_train, 13)
Y_test = np_utils.to_categorical(Y_test, 13)
print(Y_train[8])
print(train.head(50))

In [ ]:
margin = 0.3

In [ ]:
def custum_mean_squared_error(y_true, y_pred):
    return K.mean(K.square(((y_pred - y_true)))+(y_pred - y_true)*margin, axis=-1)

**THE LSTM NetWork**

In [ ]:
n_hidden = 150
batch_size = 1500
n_epoch = 1
left_input = Input(shape=(x_train_left.shape[1], x_train_left.shape[2]))
right_input = Input(shape=(x_train_left.shape[1], x_train_left.shape[2]))
shared_lstm = LSTM(n_hidden)

left_out = shared_lstm(left_input)
right_out = shared_lstm(right_input)
left_out_norm = BatchNormalization()(left_out)
right_out_norm = BatchNormalization()(right_out)
merged_vector = concatenate([left_out_norm, right_out_norm], axis=-1)
#malstm_distance = Lambda(function=lambda x: cossim(x[0], x[1]),output_shape=lambda x: (1, 1))([left_out, right_out])
predictions2 = Dense(20,activation="sigmoid")(merged_vector)
predictions3 = Dense(13,activation="softmax")(predictions2)
# Pack it all up into a model
malstm = Model([left_input, right_input], [predictions3])

#optimizer = Adadelta(clipnorm=1.25)
#malstm.compile(loss='mse', optimizer="adam", metrics=['accuracy'])
#sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

malstm.compile(loss='mse', optimizer="adam", metrics=['accuracy'])
malstm.summary()
# Start training
training_start_time = time()
malstm_trained = malstm.fit([x_train_left, x_train_right],Y_train, batch_size=batch_size, nb_epoch=n_epoch,validation_split=0.05)

**Good One:**

In [ ]:
val = int(math.floor(until))+X_test.shape[0]
x_test_left = np.empty([X_test.shape[0],60,lenthofchar])
for i in range(int(math.floor(until))+1,val):
    lis =(list(X_test.loc[i]["product_title"]))
    for j in range(60):
        if (j<len(lis)):
            if ([lis[j]]!='\xa0'):
                x_test_left[i-int(math.floor(until))-1][j] = CreateOneHot(chars.find(lis[j]))
            else:
                x_test_left[i-int(math.floor(until)-1)][j] =  CreateOneHot(chars.find("$")) 
x_test_right = np.empty([X_test.shape[0],60,lenthofchar])
for i in range(int(math.floor(until))+1,val):
    lis2 =(list(X_test.loc[i]["search_term"]))
    for j in range(60):
        if (j<len(lis2)):
            if ([lis2[j]]!='\xa0'):
                x_test_right[i-int(math.floor(until))-1][j] = CreateOneHot(chars.find(lis2[j]))
            else:
                x_test_right[i-int(math.floor(until))-1][j] = CreateOneHot(chars.find("$"))

In [ ]:
preds = malstm.predict([x_test_left, x_test_right], batch_size=batch_size)

In [ ]:
rmse = np.sqrt(mean_squared_error(preds, Y_test))
MAE = mean_absolute_error(preds, Y_test)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % MAE)
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('model loss')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()
aa=[x for x in range(100)]
plt.plot(aa, Y_test[:100], marker='.', label="actual")
plt.plot(aa, preds[:100], 'r', label="prediction")
plt.ylabel('Relevance', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show()